In [2]:
import numpy as np
import scipy.io.wavfile

freq_1 = 440
freq_2 = 900
x = np.linspace(0, 2*np.pi, 10000)
y_1 = np.sin(freq_1*x)
y_2 = np.sin(freq_2*x)
signal = np.sin(x)**2 * y_1 + np.cos(x)**2 * y_2

In [15]:
import os
import os.path
import scipy.signal

def comb_filter(length_ms, amplitude, delay_ms, sample_rate=16000):
    delay = int(sample_rate * delay_ms / 1000)
    signal = np.zeros((sample_rate * length_ms / 1000, ))
    signal[40] = 1
    for i in range(40, sample_rate * length_ms / 1000, delay):
        delay_i = i + delay
        if delay_i >= sample_rate * length_ms / 1000:
            break
        signal[delay_i] = signal[i] * amplitude
    return signal

def echo(length_ms, amplitude, delay_ms, sample_rate=16000):
    delay = int(sample_rate * delay_ms / 1000)
    signal = np.zeros((sample_rate * length_ms / 1000, ))
    signal[40] = 1
    signal[delay] = amplitude
    return signal

def silence(length=None, samples=None, db=-150, sample_rate=16000):
    if length:
        samples = length*sample_rate
    x = np.random.normal(size=(samples,))
    factor = np.power(10, db / 20.0)
    y = x * factor
    return y

def get_h(h_type, normalise=True):
    if h_type == 'short':
        h = echo(200, 0.7, 40)
    if h_type == 'long':
        h = echo(200, 0.7, 170)
    if h_type == 'excessive':
        h = echo(200, 0.7, 190)
    if h_type == 'decaying':
        h = comb_filter(500, -0.9, 12)
    if normalise:
        h = h / np.sum(np.abs(h))
    return h
    raise Exception("H type '%s' not valid" % h_type)
    
def get_near_end(length, frequencies=[440], sample_rate=16000, rshift=4):
    x = np.linspace(0, length * 2 * np.pi, length * sample_rate)
    signal = np.zeros((length * sample_rate,))
    for freq in frequencies:
        signal += np.sin(freq * x)
    return signal / (1<<rshift)

def get_ref_discrete(length, freq_a=1000, freq_b=2000, period=1, sample_rate=16000, rshift=4):
    x = np.linspace(0, length * 2 * np.pi, length * sample_rate)
    y_1 = np.sin(freq_a * x)
    y_2 = np.sin(freq_b * x)
    signal = np.sin(x / (period*2))**2 * y_1 + np.cos(x / (period*2))**2 * y_2
    return signal / (1<<rshift)

def get_ref_continuous(length, freq_a=500, freq_b=4000, period=0.2, sample_rate=16000, rshift=4):
    x = np.linspace(0, length * 2 * np.pi, length * sample_rate)
    f = np.sin(x/period)*(freq_b-freq_a)/2 + (freq_a+freq_b)/2
    #f = np.tile(scipy.signal.triang(sample_rate * period), length/period + 1)[:len(x)]*(freq_b-freq_a)/2 + (freq_a+freq_b)/2
    y = np.cumsum(f) / sample_rate * 2 * np.pi
    z = np.sin(y)
    signal = z
    return signal / (1<<rshift)

def get_ref(*args, **kwargs):
    return get_ref_continuous(*args, **kwargs)

def write_data(data, filename, sample_rate=16000, dtype=np.int16, rshift=0):
    output = np.asarray(data*np.iinfo(np.int16).max, dtype=dtype) >> rshift
    scipy.io.wavfile.write(filename, sample_rate, output.T)

def write_audio(test_class, echo, AudioIn, AudioRef, sample_rate=16000, dtype=np.int16):
    audio_dir = 'spec_audio'
    filename = '%s-%s-%s' % (test_class, echo, "%s")
    try:
        os.makedirs(audio_dir)
    except os.error:
        pass
    write_data(AudioIn, os.path.join(audio_dir, filename % "AudioIn.wav"), sample_rate, dtype)
    write_data(AudioRef, os.path.join(audio_dir, filename % "AudioRef.wav"), sample_rate, dtype)

In [16]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import scipy.signal

length=10; freq_a=2000; freq_b=3000; period=2; sample_rate=16000; rshift=8

x = np.linspace(0, length * 2 * np.pi, length * sample_rate)
f = np.sin(x/period)*(freq_b-freq_a)/2 + (freq_a+freq_b)/2
#f = np.tile(scipy.signal.triang(sample_rate * period), length/period + 1)[:len(x)]*(freq_b-freq_a)/2 + (freq_a+freq_b)/2
x = np.cumsum(f) / sample_rate * 2 * np.pi
plt.plot(x)
plt.show()
y = np.sin(x)
signal = y
plt.plot(f)
plt.show()
plt.plot(y)
plt.show()
plt.specgram(signal, Fs=16000)
plt.show()

In [17]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#plt.magnitude_spectrum(signal, Fs=10000)
#signal = get_ref(2, 440, 2000, period=0.5)
signal = get_ref(10)
plt.specgram(signal, Fs=16000)
plt.show()
signal = get_near_end(1, [440])
plt.specgram(signal, Fs=16000)
plt.show()

h = get_h('short')
x = np.linspace(0, len(h)/16000.0, len(h))
plt.plot(x, h)
plt.show()
h = get_h('long')
x = np.linspace(0, len(h)/16000.0, len(h))
plt.plot(x, h)
plt.show()
h = get_h('excessive')
x = np.linspace(0, len(h)/16000.0, len(h))
plt.plot(x, h)
plt.show()
h = get_h('decaying')
x = np.linspace(0, len(h)/16000.0, len(h))
plt.plot(x, h)
plt.show()
y = silence(60)
x = np.linspace(0, len(y)/16000.0, len(y))
plt.plot(x, y)
plt.show()
print y
write_data(y, "silence.wav")
write_data(signal, "test.wav")

In [18]:
import scipy.signal

ref = get_ref(2, 440, 2000, period=0.5)
AudioIn = scipy.signal.convolve(ref, get_h('decaying'))
# plt.plot(ref)
# plt.show()
# plt.plot(AudioIn)
# plt.show()

ref = get_ref(2, 1000, 3000, period=0.5)
ref_2f = get_ref_discrete(2, 1000, 3000, period=0.5)
plt.figure(figsize=(8,3))
plt.subplot(121)
plt.specgram(ref, Fs=16000)
plt.title("Continuous 2-tone Reference")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.subplot(122)
plt.specgram(ref_2f, Fs=16000)
plt.title("Discrete 2-tone Reference")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.tight_layout()
plt.savefig("reference.png")
plt.show()

In [19]:
# Generate Audio
from scipy.signal import convolve
# Simple Tests: Short Echo
ref = get_ref_discrete(20)
ref_room = convolve(ref, get_h('short'))
background_noise = silence(samples=len(ref_room))
near_end = get_near_end(10)
near_end = np.concatenate((background_noise[:-len(near_end)], background_noise[-len(near_end):] + near_end))
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('simple', 'short', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()

# Simple Tests: Long Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('long'))
background_noise = silence(samples=len(ref_room))
near_end = get_near_end(60)
near_end = np.concatenate((background_noise[:-len(near_end)], background_noise[-len(near_end):] + near_end))
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('simple', 'long', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)  
plt.show()

# Simple Tests: Decaying Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('decaying'))
background_noise = silence(samples=len(ref_room))
near_end = get_near_end(60)
near_end = np.concatenate((background_noise[:-len(near_end)], background_noise[-len(near_end):] + near_end))
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('simple', 'decaying', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()

In [20]:
# Multi-tone Tests: Short Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('short'))
background_noise = silence(samples=len(ref_room))
near_end = convolve(get_near_end(60, frequencies=[500, 1500, 3000]), get_h('short'))
near_end = np.concatenate((background_noise[:-len(near_end)], background_noise[-len(near_end):] + near_end))
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('multitone', 'short', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()

# Multi-tone Tests: Long Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('long'))
background_noise = silence(samples=len(ref_room))
near_end = get_near_end(60, frequencies=[500, 1500, 3000])
near_end = np.concatenate((background_noise[:-len(near_end)], background_noise[-len(near_end):] + near_end))
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('multitone', 'long', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()

# Multi-tone Tests: Decaying Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('decaying'))
background_noise = silence(samples=len(ref_room))
near_end = get_near_end(60, frequencies=[500, 1500, 3000])
near_end = np.concatenate((background_noise[:-len(near_end)], background_noise[-len(near_end):] + near_end))
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('multitone', 'decaying', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()

In [21]:
sample_rate = 16000
# Impulse Response Tests: Short Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('short'))
background_noise = silence(samples=len(ref_room))
near_end = background_noise
ref_room[60*sample_rate:] = 0
near_end[60*sample_rate] = -1
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('impulseresponse', 'short', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()

# Impulse Response Tests: Long Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('long'))
background_noise = silence(samples=len(ref_room))
near_end = background_noise
ref_room[60*sample_rate:] = 0
near_end[60*sample_rate] = -1
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('impulseresponse', 'long', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()

# Impulse Response Tests: Decaying Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('decaying'))
background_noise = silence(samples=len(ref_room))
near_end = background_noise
ref_room[60*sample_rate:] = 0
near_end[60*sample_rate] = -1
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('impulseresponse', 'decaying', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()

In [22]:

# Simple Tests: Short Echo
ref = get_ref(120)
ref_room = convolve(ref, get_h('excessive'))
background_noise = silence(samples=len(ref_room))
near_end = get_near_end(60)
near_end = np.concatenate((background_noise[:-len(near_end)], background_noise[-len(near_end):] + near_end))
AudioIn = ref_room + near_end
AudioRef = ref_room
write_audio('excessive', 'excessive', AudioIn, AudioRef)

plt.specgram(AudioIn, Fs=16000)
plt.show()